# RAG 进阶：使用 Rerank（重排）优化检索精度

## 1. 为什么需要重排？

在基础的 RAG 流程中，我们通常依赖 **向量检索（Vector Search）**。虽然向量检索速度极快，但它本质上是基于语义空间的“模糊匹配”，有时会召回一些语义相近但事实不相关的干扰文档。

**重排（Reranking）** 就像是给检索流程加了一个“精算师”：
1. **召回（Recall）**：从海量文档中快速筛选出最相关的 50-100 个文档（速度快，精度中等）。
2. **重排（Rerank）**：对这 100 个文档进行深度比对，重新打分排序（速度较慢，精度极高）。

本教程将带你使用 **硅基流动 (SiliconFlow)** 提供的 `BAAI/bge-reranker-v2-m3` 模型实现这一过程。

## 2. 环境准备

首先，我们需要安装必要的异步 HTTP 客户端库和环境变量管理工具。

In [1]:
!pip install aiohttp python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.mirrors.ustc.edu.cn/simple


请确保你在当前目录下创建了 `.env` 文件，并配置了你的 API Key：
```text
OPENAI_API_KEY=sk-xxxx... (你的硅基流动 API Key)
```

## 3. 实现重排逻辑

我们将模拟一个典型的场景：用户提出了一个关于“人工智能”的问题，而我们的向量库召回了 5 条信息，其中包含一些无关的“噪音”（如苹果、天气）。

In [2]:
import asyncio
import os
import aiohttp
from dotenv import load_dotenv

# 加载配置
load_dotenv()

async def test_reranker():
    # 1. 配置 API 信息
    api_key = os.getenv("OPENAI_API_KEY")
    api_base = "https://api.siliconflow.cn/v1/rerank"
    
    # 2. 准备测试数据（模拟向量检索召回的结果）
    query = "什么是人工智能？"
    passages = [
        "人工智能是计算机科学的一个分支，它企图了解智能的实质，并生产出一种新的能以人类智能相似的方式做出反应的智能机器。",
        "苹果是一种水果，富含维生素和纤维，有助于健康。",
        "机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并做出预测或决策。",
        "今天天气很好，适合户外活动。",
        "深度学习是机器学习的一个分支，它模仿人脑神经网络的工作方式。"
    ]
    
    print(f"【查询词】: {query}\n")
    print("【原始文档顺序（召回结果）】:")
    for i, passage in enumerate(passages):
        print(f"{i+1}. {passage}")
    
    # 3. 构造 Rerank 请求体
    payload = {
        "model": "BAAI/bge-reranker-v2-m3",
        "query": query,
        "documents": passages,
        "instruction": "请根据查询对文档进行相关性重排序",
        "top_n": len(passages),
        "return_documents": True
    }
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # 4. 发送异步请求
    async with aiohttp.ClientSession() as session:
        async with session.post(api_base, json=payload, headers=headers) as response:
            if response.status == 200:
                result = await response.json()
                
                ranked_results = []
                if 'results' in result:
                    for item in result['results']:
                        doc_text = item['document']
                        if isinstance(doc_text, dict): doc_text = doc_text.get('text', '')
                        score = item['relevance_score']
                        ranked_results.append((doc_text, score))
                
                # 5. 展示重排后的结果
                print("\n【重排后的结果 (基于 BGE-Reranker-V2-M3)】:")
                for i, (passage, score) in enumerate(ranked_results):
                    print(f"排名 {i+1} [得分: {score:.4f}] {passage}")
            else:
                print(f"错误: {response.status}, {await response.text()}")

# 在 Jupyter 中运行异步函数
await test_reranker()

【查询词】: 什么是人工智能？

【原始文档顺序（召回结果）】:
1. 人工智能是计算机科学的一个分支，它企图了解智能的实质，并生产出一种新的能以人类智能相似的方式做出反应的智能机器。
2. 苹果是一种水果，富含维生素和纤维，有助于健康。
3. 机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并做出预测或决策。
4. 今天天气很好，适合户外活动。
5. 深度学习是机器学习的一个分支，它模仿人脑神经网络的工作方式。

【重排后的结果 (基于 BGE-Reranker-V2-M3)】:
排名 1 [得分: 0.9974] 人工智能是计算机科学的一个分支，它企图了解智能的实质，并生产出一种新的能以人类智能相似的方式做出反应的智能机器。
排名 2 [得分: 0.2787] 机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并做出预测或决策。
排名 3 [得分: 0.1172] 深度学习是机器学习的一个分支，它模仿人脑神经网络的工作方式。
排名 4 [得分: 0.0000] 苹果是一种水果，富含维生素和纤维，有助于健康。
排名 5 [得分: 0.0000] 今天天气很好，适合户外活动。


## 4. 技术重点解析

### 1. 评分机制
在返回结果中，你会发现 `relevance_score`。这是一个衡量相关性的关键指标：
- **高分（如 0.8+）**：表示文档与问题高度匹配。
- **低分（如 0.01-）**：即使文档中包含某些关键词（如“苹果”），但语义不符，会被 Reranker 识别并降权。

### 2. 为什么选择 BGE-Reranker-V2-M3？
- **多语言能力**：对中文、英文及其他多种语言有极佳的支持。
- **交叉编码器（Cross-Encoder）**：与传统的双塔向量模型不同，它将问题和文档同时输入模型进行全注意力计算，能够捕捉到更深层的逻辑关系。

### 3. 在 RAG 工作流中的位置
```text
用户提问 -> 向量检索(Top 100) -> Reranker(Top 5) -> LLM 生成回答
```
通过在中间加入 Reranker，你可以显著减少无效背景信息对大模型的干扰，从而**降低幻觉**并**提高准确度**。

## 5. 课后练习
1. **尝试干扰**：在 `passages` 中加入一条包含“人工智能”关键词但内容完全无关的句子（例如：一则关于《人工智能》电影影评的句子），观察 Reranker 是否能将其排在科普定义之后。
2. **阈值过滤**：修改代码逻辑，只将 `score > 0.3` 的文档送给 LLM，观察这在生产环境下有什么好处。